In [3]:
import pandas as pd

In [65]:
train = pd.read_csv("train.csv")
train = train.iloc[:,:7].dropna()
train

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5
...,...,...,...,...,...,...,...
79193,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5
79194,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5
79195,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5
79196,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5


In [70]:
selfeval = train['selfeval']
train["empathy"] = selfeval.str.extract('\d\|\d\|\d_(\d)\|\d\|\d')
print(train.shape)
train = train.dropna()
print(train.shape)

(78304, 8)
(78304, 8)


In [123]:
conversations = train["conv_id"].value_counts()
conversations = conversations.iloc[:200]
print(conversations)
lsconv = []
for ids in conversations.index:
    convo = train.loc[train["conv_id"] == ids]
    convo = convo.drop(columns = ["utterance_idx", "selfeval", "conv_id"])
    s1id = convo["speaker_idx"].unique()[0]
    s2id = convo["speaker_idx"].unique()[1]
    s1 = convo.loc[convo["speaker_idx"]==s1id]
    s2 = convo.loc[convo["speaker_idx"]==s2id]
    s1words = s1["utterance"].values.tolist()
    s2words = s2["utterance"].values.tolist()
    context = convo["context"].str.extract('(\w+)').values[0].item()
    escore = int(convo["empathy"].values[0])
    lsconv.append([context, s1words, s2words, escore])


hit:6790_conv:13580     8
hit:7872_conv:15744     8
hit:11399_conv:22798    8
hit:6773_conv:13547     8
hit:5476_conv:10952     8
                       ..
hit:7463_conv:14926     8
hit:62_conv:124         8
hit:8807_conv:17615     8
hit:9787_conv:19575     8
hit:10105_conv:20211    8
Name: conv_id, Length: 200, dtype: int64


[['surprised',
  ['I found out that my wife is pregnant today.',
   'Very.  It will be my first.',
   'No clue',
   "I'm rooting for a son myself.  I can't say she feels the same."],
  ['Are you excited?',
   "Do you know if you're having a boy or girl yet?",
   "I hope it's a boy I love kids.",
   'Got to get him playing some sports for sure.'],
  4],
 ['ashamed',
  ['Earlier this year my mom got really sick and ended up in the hospital. I feel like I should have been more vigilant about her health.',
   "Yes_comma_ that's true_comma_ they won't always listen. Still I can't help but beat myself up about it_comma_ and feel like I could have done more. I'm trying to get past the feeling of guilt_comma_ but it's hard. ",
   'Thank you so much for your comforting words. At least she is doing a bit better now. I guess I have to accept that there is only so much I can do. ',
   'That is a really good point. Thanks again so much.'],
  ["So sorry to hear about your Mom - I do hope she's bette

My goal is to output a list of lists of the prompt and all that the speaker said, the listener responses, and the first empathy value

First need to take each conversation and seperate it, then need to seperate by speaker_id, and add each response to a new list